<a href="https://colab.research.google.com/github/dmswneunju/DeepLearning_signiture/blob/main/ch08_CNN_VGGNet_CIFAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VGGNet
Convolutuin layer & pooling layer : feature 추출  
Dense layer : 주어진 feature들이 어떤 카테고리에 속하는지 분류기 학습  
VGGNet의 가장 큰 특징 : 모든 레이어가 3X3 kernel을 사용,  
의의 : 커널 사이즈를 크게 설정하면 이미지의 넓은 영역에 대한 정보를 추출 가능.  But, 커널 사이즈 크게 하면 연산량이 많아짐. 
3X3 kernel은 작지만, convolution연산을 반복해서 수행하게 되면 커널사이즈를 키운 것과 동일한 효과 및 좋은 성능.  
* 시각화 관련 모델이므로 torchvision에서 model import가능
* loss : CrossEntropyLoss
* optimizer : SGD  

재활용하는 부분은 피쳐를 뽑아내는 부분. 주어진 이미지에서 피쳐를 잘 뽑아낼 수 있었기에 성능이 좋았음.  
앞부분의 피쳐 뽑는부분은 그대로 fix. classifier에 대한부분은 다시 재학습.  
즉, optimizer가 학습을 해야하는 부분도 classifier의 파라미터부분만 학습하도록 설정.

## CIFAR10 Classifier(VGGNet)

### [Step1] Load libraries & Datasets

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader
from torch import nn

from torchvision import datasets
from torchvision.transforms import transforms #Augmentation
from torchvision.transforms.functional import to_pil_image #데이터 시각화

### [Step2] Data preprocessing
불러온 이미지의 증강을 통해 학습 정확도를 향상
* RandomCrop
* RandomHorizontalFlip
* Normalize -> 이미지 안을 구성하고있는 dataset에 대해서 평균과 표준편차를 맞춰주는 역할

In [ ]:
#데이터 불러오기전에 transform까지 지정해주면 한번에 데이터전처리까지 진행해서 불러올 수 있다.
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224,224)),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)),
])

 # CIFAR 데이터 불러오기
train_img = datasets.CIFAR10(
     root = 'data',
     train = True,
     download = True,
     transform = transform
 )
test_img = datasets.CIFAR10(
     root = 'data',
     train = False,
     download = True,
     transform = transform
 )

Files already downloaded and verified
Files already downloaded and verified


### [Step3] Set hyperparameters

In [ ]:
EPOCHS = 10
BATCH_SIZE = 32
LEARNING_RATE = 1e-3
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using Device:', DEVICE)

Using Device: cuda


### [Step4] Create DataLoader

In [ ]:
train_loader = DataLoader(train_img, batch_size = BATCH_SIZE, shuffle = True) 
test_loader = DataLoader(test_img, batch_size = BATCH_SIZE, shuffle = False)

### [Step5] Set Network Structure

In [ ]:
# Model
cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

class VGG(nn.Module):
    def __init__(self, vgg_name):
        super(VGG, self).__init__()
        self.features = self._make_layers(cfg[vgg_name])
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 360),
            nn.ReLU(inplace=True),#inplace 연산은 결과값을 새로운 변수에 값을 저장하는 대신 기존의 데이터를 대체
            nn.Dropout(),
            nn.Linear(360, 100),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(100, 10),
        )

    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out

    def _make_layers(self, cfg):
        layers = []
        in_channels = 3
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.BatchNorm2d(x),
                           nn.ReLU(inplace=True)]
                in_channels = x
        return nn.Sequential(*layers)


### [Step6] Create Model instance

In [ ]:
model = VGG('VGG16').to(DEVICE)
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

### [Step7] Model compile

In [ ]:
# loss
loss = nn.CrossEntropyLoss() #다중 분류
# Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9)

### [Step8] Set train loop

In [ ]:
def train(train_loader, model, loss_fn, optimizer):
    model.train()
    
    size = len(train_loader.dataset)
    
    for batch, (X, y) in enumerate(train_loader):
        X, y = X.to(DEVICE), y.to(DEVICE)
        pred = model(X)

        # 손실 계산
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f'loss: {loss:>7f}  [{current:>5d}]/{size:5d}')

### [Step9] Set test loop

In [ ]:
def test(test_loader, model, loss_fn):
    model.eval()

    size = len(test_loader.dataset)
    num_batches = len(test_loader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in test_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:8f}\n")

### [Step10] Run model

In [ ]:
for i in range(5):
  print(f'EPOCHS {i+1} \n------------------')
  train(train_loader, model, loss, optimizer)
  test(test_loader, model, loss)
print('Done!')

EPOCHS 1 
------------------


/usr/local/lib/python3.9/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


loss: 2.330682  [    0]/50000
loss: 1.991845  [ 3200]/50000
loss: 2.123824  [ 6400]/50000
loss: 1.852248  [ 9600]/50000
loss: 1.694799  [12800]/50000
loss: 1.503081  [16000]/50000
loss: 1.806256  [19200]/50000
loss: 1.576928  [22400]/50000
loss: 1.736798  [25600]/50000
loss: 1.568263  [28800]/50000
loss: 1.635224  [32000]/50000
loss: 1.576840  [35200]/50000
loss: 1.360161  [38400]/50000
loss: 1.026262  [41600]/50000
loss: 0.993501  [44800]/50000
loss: 1.260544  [48000]/50000
Test Error: 
 Accuracy: 51.8%, Avg loss: 1.283257

EPOCHS 2 
------------------
loss: 1.740008  [    0]/50000
loss: 1.005345  [ 3200]/50000
loss: 1.169857  [ 6400]/50000
loss: 1.574085  [ 9600]/50000
loss: 1.142755  [12800]/50000
loss: 1.110011  [16000]/50000
loss: 0.973142  [19200]/50000
loss: 1.197423  [22400]/50000
loss: 1.049568  [25600]/50000
loss: 1.116084  [28800]/50000
loss: 1.420144  [32000]/50000
loss: 0.853115  [35200]/50000
loss: 0.873093  [38400]/50000
loss: 0.838794  [41600]/50000
loss: 0.849791  [448

## CIFAR Classifier(Pretrained VGGNet)
ImageNet 데이터로 학습한 VGGNet을 사용하여 주어진 데이터 셋에서 사용할 수 있도록 Fine tuning

In [ ]:
from torchvision import models

vgg16 = models.vgg16(pretrained=True)
vgg16.to(DEVICE)
print(vgg16)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
vgg16.classifier[6].out_features=10

for param in vgg16.features.parameters():
  param.requires_grad=False #features를 구성하는 파라미터에 대해서 학습 진행 x

In [ ]:
# loss
loss = nn.CrossEntropyLoss() #다중 분류
# Optimizer
optimizer = torch.optim.SGD(vgg16.parameters(), lr=LEARNING_RATE, momentum=0.9)

In [ ]:
for i in range(5):
  print(f'EPOCHS {i+1} \n------------------')
  train(train_loader, vgg16, loss, optimizer)
  test(test_loader, vgg16, loss)
print('Done!')

EPOCHS 1 
------------------
loss: 13.333236  [    0]/50000
loss: 1.046628  [ 3200]/50000
loss: 1.079368  [ 6400]/50000
loss: 0.819446  [ 9600]/50000
loss: 0.378889  [12800]/50000
loss: 0.811578  [16000]/50000
loss: 1.032740  [19200]/50000
loss: 0.579613  [22400]/50000
loss: 0.679237  [25600]/50000
loss: 0.586219  [28800]/50000
loss: 0.671203  [32000]/50000
loss: 0.503260  [35200]/50000
loss: 0.378627  [38400]/50000
loss: 0.760421  [41600]/50000
loss: 0.565480  [44800]/50000
loss: 0.810816  [48000]/50000
Test Error: 
 Accuracy: 82.8%, Avg loss: 0.494896

EPOCHS 2 
------------------
loss: 0.795687  [    0]/50000
loss: 0.539555  [ 3200]/50000
loss: 0.309791  [ 6400]/50000
loss: 0.833226  [ 9600]/50000
loss: 0.431570  [12800]/50000
loss: 0.423443  [16000]/50000
loss: 0.430586  [19200]/50000
loss: 0.859339  [22400]/50000
loss: 0.288819  [25600]/50000
loss: 0.404751  [28800]/50000
loss: 0.378091  [32000]/50000
loss: 0.236250  [35200]/50000
loss: 0.271055  [38400]/50000
loss: 0.864372  [416